# Čtvrté cvičení - vyhlazování jazykového modelu
- Vytvořte bigramový jazykový model vyhlazený metodou Witten-Bell pro soubor TEXTEN1
- Soubor přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase
- Uvažujte na začátku věty startovací symbol <s>

In [5]:
import re, string
import numpy as np
from collections import Counter

In [6]:
with open('TEXTEN1.txt', "r") as f:
    en_text = f.read().splitlines()

In [7]:
def sentences_unigram(text):
    split_on = set('.?!')
    sentences = [['<s>']]
    for word in text:
        if re.match(r'[\w\-]+', word):
            sentences[-1].append(word.lower())
        elif word in split_on:
            sentences.append(['<s>'])
    return sentences

def unigram(sentences):
    return Counter([
        word
        for sent in sentences
        for word in sent
    ])

def sentences_bigram(text):
    return [
        sent + ['</s>']
        for sent in sentences_unigram(text)
    ]

def bigram(sentences):
    b_freq = Counter([
        (w1, w2)
        for sent in sentences
        for (w1, w2) in zip(sent[:-1], sent[1:])
    ])
    return b_freq, unigram(sentences)

def distinct(bi_model, uni_model):
    uniq = {}
    for (prev, this), count in bi_model.items():
        if prev not in uniq:
            uniq[prev] = 0
        uniq[prev] += 1
    return {
        word: uniq.get(word, 0)
        for word, count in uni_model.items()
    }

def witten_bell(bi_model, n, t):
    z = Counter({
        word: len(n) - count
        for word, count in t.items()
    })
    return Counter({
        f'{w1} {w2}':
        t[w1] / z[w1] / (c1 + t[w1])
        if bi_model[(w1, w2)] == 0
        else bi_model[(w1, w2)] / (c1 + t[w1])
        for w1, c1 in n.items()
        for w2, c2 in n.items()
    })

In [8]:
en_sent = sentences_bigram(en_text)
en_model, en_uni = bigram(en_sent)
en_uniq = distinct(en_model, en_uni)
en_bigram_model = witten_bell(en_model, en_uni, en_uniq)

print(en_bigram_model["of the"])
#0.258387066629287
print(en_bigram_model["of of"])
#1.6608937523330386e-05

KeyboardInterrupt: 